In [32]:
import os
import pandas as pd 

path = 'Data' #modify path of file directory as appropriate
items = os.listdir(path)
list_of_paths = []

for a in items:
    list_of_paths.append(path + '/' + a)

def pd_analyse_water_levels(path):
    """This function takes in the file path of a csv
    perfoms analysis with pandas
    Columns in the csv are Date and time, water_levels, 
    flag which is a categorical variable 1 or 0
    """
    ld = pd.read_csv(path)
    df = ld.iloc[:,:3]
    old_column = ld.columns[:3]

    df = df.rename(columns = {old_column[0]:'datetime', old_column[1]:'water_level', old_column[2]: 'is_high_tide'})

    df['datetime'] = pd.to_datetime(df['datetime'])
    df['water_level'] = pd.to_numeric(df['water_level'], errors= 'ignore').astype(float)
    df['month'] = df['datetime'].dt.month
    df['year'] = df['datetime'].dt.year

    df_high_tide = df[df["is_high_tide"] == 1]['water_level']
    df_low_tide = df[df["is_high_tide"] == 0]['water_level']
    summary_statistics = {'tide_high': {'mean':round(df_high_tide.mean(),2), 
                'median':round(df_high_tide.median(),2), 
                'interquartile_range':round((df_high_tide.quantile(.75) - df_high_tide.quantile(.25)),2)},
    'tide_low': {'mean':round(df_low_tide.mean()), 
                'median':round(df_low_tide.median(),2), 
                'interquartile_range':round((df_low_tide.quantile(.75) - df_low_tide.quantile(.25)),2)}}

    # Calculate ratio of high tide days
    all_high_days = df.query('is_high_tide==1').groupby('year').count()['water_level']
    high_days = df.query(f'(water_level>{df_high_tide.quantile(.75)}) & (is_high_tide==1)').groupby('year').count()['water_level']
    high_ratio = (high_days/all_high_days).reset_index()

    # Calculate ratio of low tide days
    all_low_days = df.query('is_high_tide==0').groupby('year').count()['water_level']
    low_days = df.query(f'(water_level<{df_low_tide.quantile(.25)}) & (is_high_tide==0)').groupby('year').count()['water_level']
    low_ratio = (low_days/all_low_days).reset_index()

    solution = {'river': path.split('/')[1].split('.')[0],'summary_statistics':summary_statistics, 'high_ratio': high_ratio, 'low_ratio':low_ratio}
    new = high_ratio.join(low_ratio, lsuffix = '_low_ratio')
    new.to_excel('{}_levels.xlsx'.format(path.split('/')[1].split('.')[0]))
    print(solution)
    

In [33]:
for path in list_of_paths:
    pd_analyse_water_levels(path)

{'river': '10-11_London_Bridge', 'summary_statistics': {'tide_high': {'mean': 3.32, 'median': 3.35, 'interquartile_range': 0.74}, 'tide_low': {'mean': -2, 'median': -2.41, 'interquartile_range': 0.54}}, 'high_ratio':     year  water_level
0   1911     0.032787
1   1912     0.127469
2   1913     0.186846
3   1914     0.161572
4   1915     0.219219
..   ...          ...
80  1991     0.252125
81  1992     0.265912
82  1993     0.317597
83  1994     0.357447
84  1995     0.324823

[85 rows x 2 columns], 'low_ratio':     year  water_level
0   1911     0.203463
1   1912     0.192793
2   1913     0.102985
3   1914     0.141618
4   1915     0.139818
..   ...          ...
80  1991     0.312057
81  1992     0.265912
82  1993     0.252496
83  1994     0.252482
84  1995     0.246809

[85 rows x 2 columns]}
{'river': '15_Richmond', 'summary_statistics': {'tide_high': {'mean': 3.72, 'median': 3.76, 'interquartile_range': 0.77}, 'tide_low': {'mean': 0, 'median': 0.04, 'interquartile_range': 1.04}}, '

MAKE INTO TIMESERIES FORECASTING